## Fisher’s Exact Test (費雪爾正確概率檢定)

[參考資料1](https://math.nsysu.edu.tw/var/file/183/1183/img/495/h.pdf)\
[參考資料2](https://highscope.ch.ntu.edu.tw/wordpress/?p=73478)\
[參考資料3](https://blog.pulipuli.info/2017/05/fishers-exact-test-example.html)\
[參考資料4](https://www.yongxi-stat.com/fishers-exact-test-r/)\
[wiki](https://en.wikipedia.org/wiki/Fisher%27s_exact_test)

<font color = red size = 5>若單元格的期望次數小於5，則使用卡方效果不佳，此時會採用 Fisher's Exact Test。</font>

<font color =red size = 5 >目的 : 分析兩組數據是否顯著相關</font>

以下皆已此表為例

|           | 組別A   |  組別B    | 合計  |
| --------- |:-------:| ---------:|------:|
| 事件1     | a       |     b     | a + b |
| 事件2     | c       |     d     | c + d |
| 合計      | a + c   |   b + d   | N     |

### 機率計算方式

此方法會<font color = red>固定所有合計欄位</font> (各事件和組別的合計欄位，其值為分析目標當前的狀態)，在此情況下，我們假設所有單元格在固定事件或組別的情況下，其發生機率皆相同 (eg. P(組別A事件1|組別A)=P(組別A事件2|組別A) 或 P(組別A事件1|事件1)=P(組別B事件1|事件1))，此時可以計算所有可能發生的機率。

因固定所有合計欄位，所以改變一個單元格的值則其他單元格也會跟著改變（換句話說只要給定一個單元格值就好，並且機率值也只要算一個單元格的機率即可），所以以 $P(A)$ 的A表示組別A事件1的單元格的值，而其發生機率為 : $\frac{\tbinom{a+b}{A}\tbinom{c+d}{a+c-A}}{\tbinom{N}{a+c}}$, 其中 $A\in [min\{a+b,a+c\}-min\{a+b,a+c,c+d,b+d\},\min\{a+b,a+c\}], A \in Z^+$

#### P value
計算出來的所有 P 值，可以從中得到原始觀察次數的機率值，在依照單雙尾檢定 (可從$H_0,H_1$看) 去把其他小於觀察次數機率值的機率值 (或稱極端值) 加總，該值就是卡方的 p_value。

#### 例子1 單尾

某疾病的死亡率如下
|           |男性   |  女性    | 合計  |
| --------- |:-------:| ---------:|------:|
| 存活     | a = 9       |     b = 4    | a + b = 13|
| 死亡     | c = 1      |     d  = 10   | c + d = 11|
| 合計      | a + c = 10   |   b + d = 14  | N  = 24    |

問:在該疾病女性的死亡率是否比男性高

##### python

In [71]:
import numpy as np

### ipmut
a = 9
ab = 13
ac = 10
bd = 14
cd = 11

### function

#階層
def factorial(x):
    if x > 0 :
        return x * factorial(x-1)
    else:
        return 1

#組合
def comb(a,b):
    if b>a:
        return "b>a"
    else:
        return factorial(a)/(factorial(b)*factorial(a-b))

### run
N = ab+cd #總數
max_a = min(ab,ac) #a 的上限
min_a = min(ab,ac) - min(ab,ac,bd,cd) #a 的下限

prob = {}
den = comb(N,ac) #分母都相同 不用再算一次
for i in range(min_a,max_a+1):
    prob[str((i,ab-i,ac-i,cd-ac+i))] = comb(ab,i)*comb(cd,ac-i)/den

print("全部可能的機率為 : ")
for i in prob:
    t = i.split(',')[0][1:]
    if int(t) == a :
        ind = int(t)
        print(i, ':', round(prob[str(i)],5) ," <---- 觀察值")
    else:
        print(i, ':', round(prob[str(i)],5) )
print()

if ind - min_a < ind - max_a:
    print( "p_value : ", np.sum( list(prob.values())[min_a:(ind+1)]  ))
else:
    print( "p_value : ", np.sum( list(prob.values())[ind:(max_a+1)]  ))


全部可能的機率為 : 
(0, 13, 10, 1) : 1e-05
(1, 12, 9, 2) : 0.00036
(2, 11, 8, 3) : 0.00656
(3, 10, 7, 4) : 0.04812
(4, 9, 6, 5) : 0.16843
(5, 8, 5, 6) : 0.30317
(6, 7, 4, 7) : 0.28873
(7, 6, 3, 8) : 0.14437
(8, 5, 2, 9) : 0.03609
(9, 4, 1, 10) : 0.00401  <---- 觀察值
(10, 3, 0, 11) : 0.00015

p_value :  0.004156010230179028


#### 例子2 雙尾

兩班考試結果
|           |甲班   |  乙班    | 合計  |
| --------- |:-------:| ---------:|------:|
| 通過     | a = 8       |     b = 1    | a + b = 9|
| 未通過     | c = 14      |     d  = 3   | c + d = 17|
| 合計      | a + c = 22   |   b + d = 4  | N  = 26    |

問:兩班通過比例是否不同

In [85]:
import numpy as np

### ipmut
a = 6
ab = 9
ac = 22
bd = 4
cd = 17

### function

#階層
def factorial(x):
    if x > 0 :
        return x * factorial(x-1)
    else:
        return 1

#組合
def comb(a,b):
    if b>a:
        return "b>a"
    else:
        return factorial(a)/(factorial(b)*factorial(a-b))

### run
N = ab+cd #總數
max_a = min(ab,ac) #a 的上限
min_a = min(ab,ac) - min(ab,ac,bd,cd) #a 的下限

prob = {}
den = comb(N,ac) #分母都相同 不用再算一次
for i in range(min_a,max_a+1):
    prob[str((i,ab-i,ac-i,cd-ac+i))] = comb(ab,i)*comb(cd,ac-i)/den

print("全部可能的機率為(a,b,c,d) : ")
for i in prob:
    t = i.split(',')[0][1:]
    if int(t) == a :
        pval = prob[str(i)]
        print(i, ':', round(prob[str(i)],5) ," <---- 觀察值")
    else:
        print(i, ':', round(prob[str(i)],5) )
print()

sum = 0
for i in prob:
    if prob[str(i)] <= pval:
        sum += prob[str(i)]

print( "p_value : ", sum)

全部可能的機率為(a,b,c,d) : 
(5, 4, 17, 0) : 0.00843
(6, 3, 16, 1) : 0.09552  <---- 觀察值
(7, 2, 15, 2) : 0.32749
(8, 1, 14, 3) : 0.40936
(9, 0, 13, 4) : 0.1592

p_value :  0.10394648829431438
